In [9]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from fastai.structured import *
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from IPython.core.debugger import set_trace
from sklearn.model_selection import KFold

import os
print(os.listdir("./"))
PATH = "./"


['.ipynb_checkpoints', 'restaurant-revenue-prediction.zip', 'test.csv', 'test_predict.ipynb', 'train.csv']


e:\python\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
# parse_dates是将日期类的数据解析成日期类型
df_train = pd.read_csv(f'{PATH}train.csv', parse_dates=['Open Date'])
df_test = pd.read_csv(f'{PATH}test.csv', parse_dates=['Open Date'])

df_joined = pd.concat([df_train.drop('revenue', axis=1), df_test], axis=0)  # 合并数据集

In [26]:
def score(model, X_train, y_train, X_valid = [], y_valid = []):  # 计算得分  均方误差
    #set_trace()
    rms = sqrt(mean_squared_error(np.square(np.exp(y_train)), np.square(np.exp(model.predict(X_train)))))
    score = [rms, model.score(X_train, y_train)]
    
    if len(X_valid) != 0 and len(y_valid) != 0:
        score.append(sqrt(mean_squared_error(np.square(np.exp(y_valid)), np.square(np.exp(model.predict(X_valid))))))
        
    if model.oob_score:   
        '''
        oob_score:袋外估计(out-of-bag)，这个外是针对于bagging这个袋子而言的，
        我们知道，bagging采取的随机抽样的方式去建立树模型，那么那些未被抽取到的样本集，
        也就是未参与建立树模型的数据集就是袋外数据集，我们就可以用这部分数据集去验证模型效果，默认值为False。
        '''
        score.append(model.oob_score_)
    
    return score

n_train = df_train.shape[0]

def prcs(df, fe=[]):
#     add_datepart(df, 'Open Date')  # 这是个将日期转化成数值型的linux函数,换下面的函数
    from datetime import datetime
    df['age'] = (datetime.now() - df['Open Date']).astype('timedelta64[D]') / 365
    df = df.drop('Open Date', axis=1) # axis=1删除列
    
    if 'city' in fe:
        df = df.drop('City', axis=1)
    # Quitamos el outlier (16)  # 删除某一列有异常值的那一行
    if 'outlier' in fe:
        df = df.drop(index=16, axis=0)
    
    if 'MB' in fe:
        # No hay apenas tipo 'MB'
        df['Type'] = df['Type'].replace('MB', 'DT')
    
    if 'city_group' in fe:
        df = df.drop('City Group', axis=1)
    
    if 'dummies' in fe:
        #Get dummies
        p_cols = [ f'P{n}' for n in range(1,38)]
            
        df = pd.get_dummies(df, columns=p_cols)   # 对P1-P37列进行独热编码   其实可以用LabelEncoder
        if 'city_group' not in fe:
            df = pd.get_dummies(df, columns=['City Group'], drop_first=True)
        df = pd.get_dummies(df, columns=['Type'])
    
    #Train cats  将类别转换为数字
#     train_cats(df)  函数不可用!!下面替换
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for col in df:
        if df.dtypes[col] == 'object':
            df[col] = le.fit_transform(df[col].values)  # 润佳  我真是太厉害了
    
    
#     X, _, _ = proc_df(df, None)  # 没有缺失 用不了fastai  弃掉  用以上函数代替
    '''
    可调用proc_df函数将所有分类数据转换为整数代码
    该函数执行以下任务：
    1. 对于连续变量，它检查列是否存在缺失值；
    2. 如果列有缺失值，则创建另一列名为 columnname_na，1代表有缺失值，0代表没有缺失值；
    3. 同时，缺失值被列的中值所代替；
    4. 对于分类变量，Pandas 用-1替换缺失值。因此，proc_df 为分类变量的所有值增加了1。因此，我们0代表缺失值，而所有其他值都递增1。
    
    返回  df, y, nas = proc_df(df_raw, 'SalePrice')
    '''
    
    X = df.copy()  # 上面的函数不可用   直接copy
    
    #  日期处理直接用数值代替了 这些参数列可能是add_datepart函数生成
#     drop_cols = ['Open Year', 'Open Month', 'Open Week', 'Open Day', 'Open Dayofweek',
#        'Open Dayofyear', 'Open Is_month_end', 'Open Is_month_start',
#        'Open Is_quarter_end', 'Open Is_quarter_start', 'Open Is_year_end',
#        'Open Is_year_start']
    
#     X = X.drop(drop_cols, axis=1)
    # La columna Id no aporta nada  Id列不会添加任何内容
    if 'id' in fe:
        X = X.drop('Id', axis=1)
    
    if 'scale_open' in fe:  # 不知道干嘛的
        X['Open Elapsed'] = (X['Open Elapsed']/1000).apply(np.log)
    
    X_train = X[:n_train]
    X_test = X[n_train:]   # 把数据集合在一起又分开???
    
    return X_train, X_test

def train_cv(X, y):
    models = []
    scores = []
    
    kf = KFold(n_splits=4, random_state=12, shuffle=False)  # 拆分为k个连续折叠
    for train_index, val_index in kf.split(X):
        X_train_ = X.iloc[train_index]  # iloc行切片 loc列切片
        y_train_ = y.iloc[train_index]
        X_val_ = X.iloc[val_index]
        y_val_ = y.iloc[val_index]
        m = RandomForestRegressor(n_jobs=-1, n_estimators=100, max_features=0.5, oob_score=True)
        m.fit(X_train_, y_train_)
        models.append(m)
        scores.append(score(m, X_train_, y_train_, X_val_, y_val_))
        
    return models, np.array(scores).mean(axis=0)

def predict(models, X):
    f = 1 / len(models)
    pred = 0
    for m in models:
        pred += f * m.predict(X)
    
    return pred

In [27]:
X_train, X_test = prcs(df_joined.copy())   # 特征工程  数据处理
y_train = df_train['revenue'].copy().apply(np.log)   # 降一个指数级
print(X_train.head())
print(X_test.head())
print(y_train.head())


   Id  City  City Group  Type  P1   P2   P3   P4  P5  P6    ...      P29  P30  \
0   0    60           0     2   4  5.0  4.0  4.0   2   2    ...      3.0    5   
1   1     4           0     1   4  5.0  4.0  4.0   1   2    ...      3.0    0   
2   2    14           1     2   2  4.0  2.0  5.0   2   3    ...      3.0    0   
3   3    52           1     2   6  4.5  6.0  6.0   4   4    ...      7.5   25   
4   4    21           1     2   3  4.0  3.0  4.0   2   2    ...      3.0    5   

   P31  P32  P33  P34  P35  P36  P37        age  
0    3    4    5    5    4    3    4  19.723288  
1    0    0    0    0    0    0    0  11.136986  
2    0    0    0    0    0    0    0   6.068493  
3   12   10    6   18   12   12    6   7.167123  
4    1    3    2    3    4    3    3   9.904110  

[5 rows x 42 columns]
   Id  City  City Group  Type  P1   P2   P3   P4  P5  P6    ...     P29  P30  \
0   0    42           1     1   1  4.0  4.0  4.0   1   2    ...     3.0    0   
1   1    31           1     2 

In [28]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=150, oob_score=True, max_features=0.5)
m.fit(X_train, y_train)
score(m,X_train, y_train)

[25503611687854.47, 0.8911943504238528, 0.21315821624242037]

In [29]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.exp(predict(m, X_test)))
# df_preds.to_csv('submission0.csv', index=True, index_label='Id')  # 先不生成文件, 先注释
df_preds.head()

,Prediction
0,4.227426e+06
1,3.397814e+06
2,3.134618e+06
3,2.915595e+06
4,3.311017e+06


# 以上是生成的第一个版本  RMSE 1.92 M (Rank 1600)

# 以下是  Cross-validation y Ensembling  交叉验证 

In [32]:
models, scores = train_cv(X_train, y_train)   # 用上面写好的cv函数交叉验证
print(scores)

[2.67327558e+13 8.93193976e-01 3.83615815e+13 1.84830796e-01]


In [33]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.exp(predict(models, X_test)))
# df_preds.to_csv('submission1.csv', index=True, index_label='Id')
df_preds.head()

,Prediction
0,4.028657e+06
1,3.459114e+06
2,2.917636e+06
3,2.997049e+06
4,3.167410e+06


# 特征处理

## 删除ID列

In [34]:
X_train, X_test = prcs(df_joined.copy(), fe=['id'])

# 双重转换使分布正常
y_train = df_train['revenue'].copy().apply(np.sqrt).apply(np.log)

In [35]:
models, scores = train_cv(X_train, y_train)
print(scores)

[1.12411971e+06 8.87824238e-01 2.33226764e+06 1.87966187e-01]


In [36]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.square(np.exp(predict(models, X_test))))
# df_preds.to_csv('submission2.csv', index=True, index_label='Id')
df_preds.head()

,Prediction
0,3.909989e+06
1,3.135632e+06
2,2.946801e+06
3,2.996367e+06
4,3.014844e+06


### RMSE 1.83 M (Rank 600)

## 在分类变量中添加虚拟对象 (对P1-P37的独热处理)

In [37]:
X_train, X_test = prcs(df_joined.copy(), fe=['id', 'dummies'])

# 双重转换使分布正常
y_train = df_train['revenue'].copy().apply(np.sqrt).apply(np.log)

In [38]:
models, scores = train_cv(X_train, y_train)
print(scores)

[1.12161275e+06 8.84084122e-01 2.32751055e+06 1.35897967e-01]


In [39]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.square(np.exp(predict(models, X_test))))
# df_preds.to_csv('submission3.csv', index=True, index_label='Id')
df_preds.head()

,Prediction
0,4.094807e+06
1,3.295021e+06
2,3.207968e+06
3,3.007144e+06
4,3.019662e+06


### RMSE 1.784 M (Rank 75)

## 删除City列

In [40]:
X_train, X_test = prcs(df_joined.copy(), fe=['id', 'dummies', 'city'])

# Doble transformación para que la distribución sea Normal
y_train = df_train['revenue'].copy().apply(np.sqrt).apply(np.log)

In [41]:
models, scores = train_cv(X_train, y_train)
print(scores)

[1.15850224e+06 8.85687892e-01 2.38885301e+06 1.20374607e-01]


In [42]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.square(np.exp(predict(models, X_test))))
# df_preds.to_csv('submission4.csv', index=True, index_label='Id')
df_preds.head()

,Prediction
0,4.279887e+06
1,3.386301e+06
2,3.173654e+06
3,3.012497e+06
4,3.021679e+06


### RMSE 1.71 M (Rank 1)

## 删除City Group列

In [43]:
X_train, X_test = prcs(df_joined.copy(), fe=['id', 'dummies', 'city', 'city_group'])

# Doble transformación para que la distribución sea Normal
y_train = df_train['revenue'].copy().apply(np.sqrt).apply(np.log)

In [44]:
models, scores = train_cv(X_train, y_train)
print(scores)

[1.19341202e+06 8.77959846e-01 2.42164920e+06 1.31847341e-01]


In [46]:
df_preds = pd.DataFrame(columns=['Prediction'],index=X_test.index, data=np.square(np.exp(predict(models, X_test))))
df_preds.to_csv('submission5.csv', index=True, index_label='Id')
df_preds.head()

,Prediction
0,4.264507e+06
1,3.398174e+06
2,3.265276e+06
3,3.140421e+06
4,3.194095e+06
